In [28]:
import os
os.environ['GOOGLE_API_KEY'] = 'your-google-ai-api-key'

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-pro", temperature=0.9)

prompt = PromptTemplate.from_template("I want to open an Indian restrobar, suggest me a fancy name for it and a {cuisine} dish.")
# response = llm.invoke(prompt)

chain = prompt | llm

# Run the chain
response = chain.invoke({"cuisine": "Chinese"})

# Print the result
print(response.content)

Of course! Opening an Indian restrobar is an exciting venture. Here are some suggestions for a fancy name and a fitting Chinese dish.

### Fancy Name for an Indian Restrobar

The key to a "fancy" name is to evoke a sense of elegance, history, or modern sophistication. Here are a few options, categorized by the vibe they create:

**Option 1: Elegant & Timeless**

### **The Saffron Table**

*   **Why it works:** "Saffron" is one of the most luxurious and prized spices in the world, instantly associated with high-quality Indian cuisine (like biryani and kheer). "The Table" suggests community, dining, and a place to gather. It's simple, elegant, and easy to remember.

**Other ideas in this category:**
*   **Darbar Lounge:** "Darbar" means the royal court of a king, suggesting regal hospitality and opulence.
*   **Maison Mumbai:** "Maison" (French for 'house') adds a touch of international chic, while "Mumbai" grounds it in a vibrant, cosmopolitan Indian city.
*   **The Cinnamon Club:** Sou

In [23]:
import google.generativeai as genai

genai.configure(api_key="your-google-ai-api-key")

models = genai.list_models()

for model in models:
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thin